In [1]:
from cnn_source import Simple3DCNN
import torch
from torchvision import transforms
import torch.nn as nn
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('mps') if torch.mps.is_available() else torch.device('cpu')
#device = torch.device('cpu')
squat_model_file_1 = 'model/squat_train.pt'
#model_squat_1 = Simple3DCNN().to(device)
#model_squat_1.load_state_dict(torch.load(squat_model_file_1,map_location=device))
model_file = 'model/classifying_cnn.pt'
model = Simple3DCNN().to(device)
model.load_state_dict(torch.load(model_file,map_location=device))

/var/folders/zc/c30_9g954mg8039jzzgzxw4h0000gn/T/ipykernel_18222/3073132621.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_file,

<All keys matched successfully>

In [2]:
import cv2
import numpy as np
import torch

# 실시간 데이터 준비 함수
def prepare_realtime_data(frame, frames_per_video=15, height=640, width=480, device='cpu'):
    frames = []

    while len(frames) < frames_per_video:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # BGR -> RGB 변환
        green_frame = frame[:, :, 1]  # 초록색 채널 추출 (H, W)
        green_frame = cv2.resize(green_frame, (width, height))  # 크기 조정
        frames.append(green_frame)

    # 부족한 프레임을 0으로 패딩
    while len(frames) < frames_per_video:
        frames.append(np.zeros((height, width), dtype=np.float32))

    # numpy 배열 -> torch 텐서 변환 및 차원 조정
    frames = torch.tensor(np.array(frames), dtype=torch.float32, device=device)  # (T, H, W)
    frames = frames.unsqueeze(1)  # (T, H, W) -> (T, C, H, W)
    frames = frames.permute(1, 0, 2, 3).unsqueeze(0)  # (T, C, H, W) -> (1, T, C, H, W)

    return frames


In [3]:
# 데이터 전처리 정의 (프레임 단위로 처리)
transform = transforms.Compose([
    transforms.ToPILImage(),                  # Numpy 배열을 PIL 이미지로 변환          
    transforms.ToTensor(),
    transforms.Resize((640,480))          
])

In [5]:
import cv2
import torch
import numpy as np
import time
import mediapipe as mp

# Mediapipe Pose 초기화
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False,
                    model_complexity=1,
                    enable_segmentation=False,
                    min_detection_confidence=0.5)

# 모델 로드
model.eval()  # 평가 모드 설정

# 카메라 캡처 객체 생성
cap = cv2.VideoCapture(0)  # 첫 번째 카메라 사용

if not cap.isOpened():
    print("Error: Cannot access the camera.")
    exit()

# 카메라 해상도 설정
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# 주요 파라미터
frames_per_video = 15
height, width = 480, 640

# 카운트다운 시작
countdown = 5
while countdown > 0:
    ret, frame = cap.read()
    #frame = cv2.resize(frame,(640,480))
    if not ret:
        print("Error: Failed to capture image.")
        break

    # BGR 이미지를 RGB로 변환 후 처리
    results = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # 검은 화면 생성
    black_frame = np.zeros((480, 640, 3), dtype=np.uint8)

    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            black_frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2)),
        mp_drawing.draw_landmarks(
            frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2))

    # 화면 크기
    height, width, _ = frame.shape

    # 텍스트 설정
    text = str(countdown)
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 3
    font_thickness = 5
    color = (0, 255, 0)

    # 텍스트 중앙에 배치
    text_size = cv2.getTextSize(text, font, font_scale, font_thickness)[0]
    text_x = (width - text_size[0]) // 2
    text_y = (height + text_size[1]) // 2

    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2))

    # 텍스트 추가
    cv2.putText(frame, text, (text_x, text_y), font, font_scale, color, font_thickness)

    # 화면 표시
    cv2.imshow("Pose Estimation", frame)
    countdown -= 1
    time.sleep(1)  # 잠시 대기

# 실시간 추론 루프
print("Starting pose estimation...")
while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture image.")
        break

    # BGR 이미지를 RGB로 변환 후 처리
    results = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # 검은 화면 생성
    black_frame = np.zeros((480, 640, 3), dtype=np.uint8)

    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            black_frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2))
    if not results.pose_landmarks:
        print("no person to make pose graph")
        break
    # 실시간 데이터 준비
    frames = prepare_realtime_data(black_frame, frames_per_video, height, width, device)

    # 모델 추론
    with torch.no_grad():
        outputs = model(frames)
        predicted_class = torch.argmax(outputs).item()

    # 결과 표시
    if predicted_class == 0:
        pose_pr = "Push-up"
    elif predicted_class == 1:
        pose_pr = "Squat"
    elif predicted_class == 2:
        pose_pr = "Lunge"
    else:
        pose_pr = "Unknown"

    # 화면에 포즈 결과 표시
    cv2.putText(frame, f"Pose: {pose_pr}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # 화면 표시
    cv2.imshow("Pose Estimation", frame)

    # 'q'를 눌러 종료
    if cv2.waitKey(1) == ord('q'):
        break

# 자원 해제
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

I0000 00:00:1737595742.548884 2581157 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1
W0000 00:00:1737595742.646027 2581850 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1737595742.659634 2581857 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1737595745.514511 2581851 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


Starting pose estimation...


NotImplementedError: The operator 'aten::max_pool3d_with_indices' is not currently implemented for the MPS device. If you want this op to be added in priority during the prototype phase of this feature, please comment on https://github.com/pytorch/pytorch/issues/77764. As a temporary fix, you can set the environment variable `PYTORCH_ENABLE_MPS_FALLBACK=1` to use the CPU as a fallback for this op. WARNING: this will be slower than running natively on MPS.